# gdsclient example demonstration

This notebook exemplifies how to use the `gdsclient` Python library to operate Neo4j GDS.
It runs the FastRP+kNN end-to-end example from the GDS Manual, found [here](https://neo4j.com/docs/graph-data-science/current/end-to-end-examples/fastrp-knn-example).


## Prerequisites

Running this notebook requires a localhost Neo4j server with a recent GDS version (2.0+) installed.
We recommend using Neo4j Desktop.

Also required are the Python libraries neo4j ([Neo4j Python Driver](https://neo4j.com/docs/api/python-driver/current/index.html)) and of course the gdsclient (see README for installation instructions).

In [2]:
from neo4j import GraphDatabase
from gdsclient import Neo4jQueryRunner, GraphDataScience

# Set up driver and gds module
URI = "bolt://localhost:7687"  # Override according to your setup
driver = GraphDatabase.driver(URI)  # You might also have auth set up in your db
runner = Neo4jQueryRunner(driver)
gds = GraphDataScience(runner)

In [44]:
# Create some data in Neo4j
runner.run_query(
    """
        CREATE
         (dan:Person {name: 'Dan'}),
         (annie:Person {name: 'Annie'}),
         (matt:Person {name: 'Matt'}),
         (jeff:Person {name: 'Jeff'}),
         (brie:Person {name: 'Brie'}),
         (elsa:Person {name: 'Elsa'}),

         (cookies:Product {name: 'Cookies'}),
         (tomatoes:Product {name: 'Tomatoes'}),
         (cucumber:Product {name: 'Cucumber'}),
         (celery:Product {name: 'Celery'}),
         (kale:Product {name: 'Kale'}),
         (milk:Product {name: 'Milk'}),
         (chocolate:Product {name: 'Chocolate'}),

         (dan)-[:BUYS {amount: 1.2}]->(cookies),
         (dan)-[:BUYS {amount: 3.2}]->(milk),
         (dan)-[:BUYS {amount: 2.2}]->(chocolate),

         (annie)-[:BUYS {amount: 1.2}]->(cucumber),
         (annie)-[:BUYS {amount: 3.2}]->(milk),
         (annie)-[:BUYS {amount: 3.2}]->(tomatoes),

         (matt)-[:BUYS {amount: 3}]->(tomatoes),
         (matt)-[:BUYS {amount: 2}]->(kale),
         (matt)-[:BUYS {amount: 1}]->(cucumber),

         (jeff)-[:BUYS {amount: 3}]->(cookies),
         (jeff)-[:BUYS {amount: 2}]->(milk),

         (brie)-[:BUYS {amount: 1}]->(tomatoes),
         (brie)-[:BUYS {amount: 2}]->(milk),
         (brie)-[:BUYS {amount: 2}]->(kale),
         (brie)-[:BUYS {amount: 3}]->(cucumber),
         (brie)-[:BUYS {amount: 0.3}]->(celery),

         (elsa)-[:BUYS {amount: 3}]->(chocolate),
         (elsa)-[:BUYS {amount: 3}]->(milk)
        """
)

[]

In [34]:
# Estimate memory of doing native graph loading
node_projection = ["Person", "Product"]
relationship_projection = {
    "BUYS": {"orientation": "UNDIRECTED", "properties": "amount"}
}
result = gds.graph.project.estimate(node_projection, relationship_projection)

print(f"Required memory for native loading: {result[0]['requiredMemory']}")

Required memory for native loading: 545 KiB


In [35]:
# Do native loading
G = gds.graph.project("purchases", node_projection, relationship_projection)

print(f"Graph named {G.name()} projected")

Graph named purchases projected


In [7]:
# Estimate memory of running FastRP
result = gds.fastRP.mutate.estimate(
    G,
    mutateProperty="embedding",
    embeddingDimension=4,
    relationshipWeightProperty="amount",
    iterationWeights=[0.8, 1, 1, 1],
)

print(f"Required memory for running FastRP: {result[0]['requiredMemory']}")

Required memory for native loading: 2224 Bytes


In [36]:
# Run FastRP and mutate projection with result
result = gds.fastRP.mutate(
    G,
    mutateProperty="embedding",
    embeddingDimension=4,
    relationshipWeightProperty="amount",
    iterationWeights=[0.8, 1, 1, 1],
)

print(f"Embedding vectors produced: {result[0]['nodePropertiesWritten']}")

Embedding vectors produced: 13


In [37]:
# Run kNN and write back to db (skip mem. estimation this time...)
result = gds.beta.knn.write(
    G,
    topK=2,
    nodeWeightProperty="embedding",
    writeRelationshipType="SIMILAR",
    writeProperty="score",
)

print(f"Relationships produced: {result[0]['relationshipsWritten']}")
print(f"Nodes compared: {result[0]['nodesCompared']}")
print(f"Mean similarity: {result[0]['similarityDistribution']['mean']}")

Relationships produced: 26
Nodes compared: 13
Mean similarity: 0.9288262587327224


In [42]:
# Rejoice in relationship in the Neo4j database!
query = """
        MATCH (p1:Person)-[r:SIMILAR]->(p2:Person)
        RETURN p1.name AS person1, p2.name AS person2, r.score AS similarity
        ORDER BY similarity DESCENDING, person1, person2
        """

# Print the most similar person nodes
runner.run_query(query)

[{'person1': 'Dan', 'person2': 'Elsa', 'similarity': 0.9828869104385376},
 {'person1': 'Elsa', 'person2': 'Dan', 'similarity': 0.9828869104385376},
 {'person1': 'Brie', 'person2': 'Matt', 'similarity': 0.9742093682289124},
 {'person1': 'Matt', 'person2': 'Brie', 'similarity': 0.9742093682289124},
 {'person1': 'Jeff', 'person2': 'Dan', 'similarity': 0.87209552526474}]

In [46]:
# Clean up database
runner.run_query("MATCH (n) DETACH DELETE n")
# Clean up graph catalog
G.drop()

[]